In [10]:
%load_ext autoreload
%autoreload 2

In [1]:
from product import Product
from json import load

with open('agora_hack_products.json', encoding='utf-8') as file:
    all_products = load(file)

all_products = [Product(**p) for p in all_products]

In [2]:
references = [p for p in all_products if p.is_reference]
references_id_set = set([ref.product_id for ref in references])
products = [p for p in all_products if p.product_id not in references_id_set]

In [3]:
from sklearn.model_selection import train_test_split

# удалим часть эталонов, что бы в датасете были 'ничейные' товары
references, nulled_references = train_test_split(
    references, test_size=0.2, random_state=42
)

nulled_references_set = set([r.product_id for r in nulled_references])
for p in products:
    if p.reference_id in nulled_references_set:
        p.reference_id = None
references.append(Product(product_id="null",
                          name="null",
                          props=[],
                          is_reference=True,
                          reference_id=""))

products_train, products_test = train_test_split(
    products, test_size=0.4, random_state=42
)
all_products = products_train + references

In [26]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import RidgeClassifier
from spacy.lang.en import STOP_WORDS as EN_STOP_WORDS
from spacy.lang.ru import STOP_WORDS as RU_STOP_WORDS

from model import ProductMatchingModel

model = ProductMatchingModel(RidgeClassifier(alpha=1.9), RidgeClassifier(alpha=1.9),
                             TfidfVectorizer(stop_words=RU_STOP_WORDS or EN_STOP_WORDS))

In [27]:
model.fit(all_products)

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "C:\Users\densh\.conda\envs\clip\lib\site-packages\IPython\core\interactiveshell.py", line 3397, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\densh\AppData\Local\Temp\ipykernel_13288\4291667864.py", line 1, in <cell line: 1>
    model.predict(products_test)
  File "C:\Users\densh\DataspellProjects\hack\agora\model.py", line 47, in predict
    unknowns = self.unknown_classifier.predict(x)
  File "C:\Users\densh\.conda\envs\clip\lib\site-packages\sklearn\linear_model\_ridge.py", line 1208, in predict
    return super().predict(X)
  File "C:\Users\densh\.conda\envs\clip\lib\site-packages\sklearn\linear_model\_base.py", line 447, in predict
    scores = self.decision_function(X)
  File "C:\Users\densh\.conda\envs\clip\lib\site-packages\sklearn\linear_model\_base.py", line 429, in decision_function
    X = self._validate_data(X, accept_sparse="csr", reset=False)
  File "C:\Users\densh\.conda\envs\clip\lib\site-p